# Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
from plotly.subplots import  make_subplots
from datetime import timedelta as td
from datetime import datetime as dt
from hbt_tools import utils_hbt as uh

# Functions

In [ ]:
def make_df_betrieb(df_pw, on_name='On Time', off_name='Off Time'):
    # On time
    time_on = df_pw[on_name]
    time_on = time_on.sort_values()
    time_on2 = time_on - td(milliseconds=1)
    time_on = pd.DataFrame(index=time_on)
    time_on.index.rename(name='zeit', inplace=True)
    time_on['betrieb'] = 1

    time_on2 = pd.DataFrame(index=time_on2)
    time_on2.index.rename(name='zeit', inplace=True)
    time_on2['betrieb'] = 0

    # Off time
    time_off = df_pw[off_name]
    time_off = time_off.sort_values()
    time_off2 = time_off + td(milliseconds=1)
    time_off = pd.DataFrame(index=time_off)
    time_off.index.rename(name='zeit', inplace=True)
    time_off['betrieb'] = 1

    time_off2 = pd.DataFrame(index=time_off2)
    time_off2.index.rename(name='zeit', inplace=True)
    time_off2['betrieb'] = 0

    # Conatenate
    df_betrieb = pd.concat([time_on2, time_on, time_off, time_off2])
    df_betrieb.sort_index(inplace=True)

    # Compute 'betriebsdauer' / 'pumpenstunden'
    df_betrieb['betriebdauer'] = df_betrieb.index
    df_betrieb['betriebdauer'] -= df_betrieb['betriebdauer'].shift(1)

    mask = df_betrieb['betrieb']==0
    df_betrieb.loc[mask,'betriebdauer'] = td()

    df_betrieb['pumpenstunden'] = df_betrieb['betriebdauer'].cumsum()
    df_betrieb['pumpenstunden'] = df_betrieb['pumpenstunden'].dt.total_seconds() / (60**2)

    return df_betrieb

# Parameters

In [ ]:
dt_format = '%d.%m.%Y %H:%M'
save_dir = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Pumpenstunden")
path_hirsen = Path(r"\\SRVHBTFS01\ProjekteK\8000-\8500er\8596"
    r"\8596.36-Mühlestrasse\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Pumpenstunden\PWHIRS 01.10.2021-31.07.2023.xlsx")
path_sonnenberg = Path(r"\\SRVHBTFS01\ProjekteK\8000-\8500er\8596"
    r"\8596.36-Mühlestrasse\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Pumpenstunden\PWSONB 01.10.2023-31.07.2023.xlsx")

date_from = dt(2021, 10, 1)
date_to = dt(2022, 9, 30)

# Read Data

## Pumpen

In [ ]:
# PW Hirsen
df_hirsen = pd.read_excel(path_hirsen, skiprows=10)
idx = [0, 2, 3, 5, 6, 8, 12]
cols = [df_hirsen.columns[i] for i in idx]
df_hirsen.columns
df_hirsen = df_hirsen.drop(columns=cols)
mask = (df_hirsen['On Time']>=date_from) & (df_hirsen['On Time']<date_to)
df_hirsen = df_hirsen[mask]

In [ ]:
# Hirsen Pumpen
mask = ((df_hirsen['Beschreibung']=='SPR PWHIRS WVA Abwasser NIVE BZ003 RB Niveau Pumpe 1 ein ')
    | (df_hirsen['Beschreibung']=='SPR PWHIRS WVA Abwasser NIVE BZ003 RB Niveau Pumpe 2 ein ')
    | (df_hirsen['Beschreibung']=='SPR PWHIRS WVA Abwasser NIVE BZ002 RB Niveau Pumpen aus '))
df_hirsen_pumpen = df_hirsen[mask]
df_hirsen_pumpen.set_index('On Time', inplace=True)
idx = [0, 1, 3, 4, 5]
cols = [df_hirsen_pumpen.columns[i] for i in idx]
df_hirsen_pumpen.drop(columns=cols)

In [ ]:
# Hirsen Pumpe 2
df_hirsen_p2 = df_hirsen[df_hirsen[
    'Beschreibung']=='SPR PWHIRS WVA Abwasser NIVE BZ004 RB Niveau Pumpe 2 ein ']
df_hirsen_p2 = df_hirsen_p2.dropna()
hirsen_betrieb2 = make_df_betrieb(df_hirsen_p2)

In [ ]:
# PW Sonnenbergstrasse
df_sonnenberg = pd.read_excel(path_hirsen, skiprows=10)
idx = [0, 2, 3, 5, 6, 8, 12]
cols = [df_sonnenberg.columns[i] for i in idx]
df_sonnenberg.columns
df_sonnenberg = df_sonnenberg.drop(columns=cols)
mask = (df_sonnenberg['On Time']>=date_from) & (df_sonnenberg['On Time']<date_to)
df_sonnenberg = df_sonnenberg[mask]

In [ ]:
# Sonnenberg Pumpe 1
df_sonnenberg_p1 = df_sonnenberg[df_sonnenberg[
    'Beschreibung']=='HER PWSONB WVA Abwasser NIVE BZ003 RB Niveau Pumpe 1 ein ']
df_sonnenberg_p1 = df_sonnenberg_p1.dropna()
sonnenberg_betrieb1 = make_df_betrieb(df_sonnenberg_p1)

In [ ]:
# Sonnenberg Pumpe 2
df_sonnenberg_p2 = df_sonnenberg[df_sonnenberg[
    'Beschreibung']=='HER PWSONB WVA Abwasser NIVE BZ004 RB Niveau Pumpe 2 ein ']
df_sonnenberg_p2 = df_sonnenberg_p2.dropna()
sonnenberg_betrieb2 = make_df_betrieb(df_sonnenberg_p2)

## Filterleistung

In [ ]:
path_filterleistung = Path(r"Q:\Projekte\8000-\8500er\8596"
    r"\8596.36-Mühlestrasse\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Pumpenstunden\Daten-Filterleistung.xlsx")

filterleistung = pd.read_excel(path_filterleistung, skiprows=1, index_col=0)

## Rain

In [ ]:
path_rain = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Regendaten\LUZ_2020 bis 2023_data.csv")

rain = uh.read_raindata_meteoschweiz(path_rain, abbr=True)
rain.index = rain.index.tz_localize(None)
rain = rain[(rain.index>=date_from) & (rain.index<date_to)]
rain['cumsum'] = rain.cumsum()

# Plots

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": False}], [{"secondary_y": True}],
    [{"secondary_y": True}], [{"secondary_y": True}]], cols=1, rows=4,
    subplot_titles=['Filterleistung', 'PW Hirsen', 'PW Sonnenbergstrasse', 'Niederschlag'], shared_xaxes=True)

# Row 1
fig.add_trace(go.Scatter(
    mode='markers',
    x=filterleistung.index,
    y=filterleistung.iloc[:,0],
    name='RFB 1',
    line=dict(color='red')
), secondary_y=False, row=1, col=1)

fig.add_trace(go.Scatter(
    mode='markers',
    x=filterleistung.index,
    y=filterleistung.iloc[:,1],
    name='RFB 2',
    line=dict(color='blue')
), secondary_y=False, row=1, col=1)

# Row 2
## Betrieb
fig.add_trace(go.Scatter(
    x=hirsen_betrieb1.index,
    y=hirsen_betrieb1.iloc[:,0],
    name='Betrieb Pumpe 1',
    line=dict(color='red')
), secondary_y=False, row=2, col=1)

fig.add_trace(go.Scatter(
    x=hirsen_betrieb2.index,
    y=hirsen_betrieb2.iloc[:,0],
    name='Betrieb Pumpe 2',
    line=dict(color='blue')
), secondary_y=False, row=2, col=1)

## Pumpenstunden
fig.add_trace(go.Scatter(
    x=hirsen_betrieb1.index,
    y=hirsen_betrieb1.iloc[:,2],
    name='Pumpenstunden Pumpe 1',
    line=dict(color='red', dash='dash')
), secondary_y=True, row=2, col=1)

fig.add_trace(go.Scatter(
    x=hirsen_betrieb2.index,
    y=hirsen_betrieb2.iloc[:,2],
    name='Pumpenstunden Pumpe 2',
    line=dict(color='blue', dash='dash')
), secondary_y=True, row=2, col=1)

# Row 3
fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,0],
    name='Regen',
    line=dict(color='blue')
), secondary_y=False, row=3, col=1)

fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,1],
    name='Regen kumulativ',
    line=dict(color='blue', dash='dash')
), secondary_y=True, row=3, col=1)

save_path = Path(r"Q:\Projekte\8000-\8500er\8596\8596.36-Mühlestrasse"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Pumpenstunden\Plot-kombiniert.html")
fig.write_html(save_path)